In [67]:
import pandas as pd
import numpy as np
import math
from math import exp


# Data Preprocessing

다섯개의 서로 다른 종류의 데이터가 존재한다. (data_index 1~5)

features는 작업자에게 추천해야 하는 카테고리이다. 

contrains는 작업자의 추천 변수의 추천 값이며 +- 2%의 값 안에 작업자 추천 변수 값이 부여되면 좋다. 

In [ ]:
# Your list of column names
df_feat = pd.read_csv('hkt_data/features_new.csv', encoding='euc-kr')

data_index = 1

features = df_feat.to_numpy()[:,0]
constrains = df_feat.to_numpy()[:,data_index]
targets = ['gt_LWM_STD_WGT', 'gt_MRM_WGT','gt_STD_WGT','gt_UPM_STD_WGT']

df_data = pd.read_csv('hkt_data/2301_2306_data_{}.csv'.format(data_index), encoding='euc-kr')
df = df_data[features]
df_targets = df_data[targets]


# Normalization

주어진 변수를 Normalize 해야한다. 

가장 좋은 방법은 maximum 값과 minimum 값을 기준으로 min-max scaling을 통해 모든 값을 [0,1]로 매핑하는 것이다. 

Constraint 가 중요한 작업에서는 주어진 lower bound와 upperbound를 기준으로 normalization을 해도 좋다. 

In [124]:
# Normalize columns using min-max scaling


lower_bound = constrains - constrains * 0.02
upper_bound = constrains + constrains * 0.02

for i in range(len(constrains)):
    if math.isnan(constrains[i]):
        lower_bound[i] = df.min()[features[i]]
        upper_bound[i] = df.max()[features[i]]


# df_x = (df - lower_bound ) / (upper_bound - lower_bound)
df_x = (df - df.min() ) / (df.max() - df.min())

# Score Calibration

Score 값을 정해 주어야 한다. 우리는 간단하게 gt_MRM_WGT를 score y로 정의하였다. 

우리는 원하는 target 인 gt_STD_WGT에 대해서 (i.e. y*), 역 생성 모델 p(x|y=y*) 를 통해 x를 생성할 것이다. 

In [126]:
from math import exp

temp = 0.3

a = 'gt_LWM_STD_WGT'
b = 'gt_UPM_STD_WGT'
x = 'gt_MRM_WGT'
x_star = 'gt_STD_WGT'


def calculate_score(row):
    score = row[x]
    return score

# Apply the scoring function to create the 'Y' column
df_y= df_targets.apply(calculate_score, axis=1)


# Dataset Preparation

일단 데이터는 시계열 특성을 가지고 있고, 이는 잘 정렬되어 있으므로, index 순서대로 threshold 지점으로 끊으면, 과거 데이터가 training data, 
그 이후의 데이터가 validation data가 된다.  

In [144]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

threshold = int(len(df_x) * 0.7)

x_train = df_x[:threshold]
y_train = df_y[:threshold]

x_valid = df_x[threshold:]
y_valid = df_y[threshold:]


# Convert pandas DataFrames to PyTorch tensors
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
x_valid_tensor = torch.tensor(x_valid.values, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32)
# x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)


# Define custom Dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Create DataLoader objects
batch_size = 256
train_dataset = CustomDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = CustomDataset(x_valid_tensor, y_valid_tensor)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)



# Oracle Model Training

Oracle 이라 함은 완벽한 model을 뜻한다. 실전 상황에서 사용 할 수 있는 모델 (예를 들어 공장 그 자체가 oracle 일 수 있음)을 뜻하며, 학습 과정에서는 사용해서는 안되고 오로지 validation 용으로만 사용 가능하다. Oracle 로 가장 좋은 것은 공장 자체가 modelling 되는 것이지만 현실적으로 불가능하다. 따라서 시계열 데이터의 모든 데이터를 다 주고, 이를 학습한 ML 모델을 Oracle이라고 가정한다 [1] 우리의 역생성 모델은 Oracle을 참고하지 않고 학습되어야 한다. 

[1] Trabucco, Brandon, et al. "Design-bench: Benchmarks for data-driven offline model-based optimization." International Conference on Machine Learning. PMLR, 2022.

In [128]:
# Oracle Score Function

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Random Forest Regression model for Oracle Training
oracle_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Oracle Can See Every Data
oracle_model.fit(df_x, df_y)
oracle_preds = oracle_model.predict(x_valid)
oracle_mse = mean_squared_error(y_valid, oracle_preds)

print(f"Oracle Regression Validation MSE: {oracle_mse:.4f}")

Oracle Regression Validation MSE: 0.0005


# Proxy Model Training

Proxy란 대체자를 뜻하는 단어이다. Proxy model은 Oracle model 과는 다르게 접근이 가능하다. Proxy model은 당연히 모든 데이터로 학습 하면 안되고, training data로만 학습되어야 한다. Proxy model은 불완전한 model 이고, 종종 큰 오류를 야기할 수 있다. 예를들어, Proxy model로만 의존해서 의사 결정을 하는 것은 Adversarial sample 등의 문제를 야기할 수 있다 [2]. 따라서 참고로만 사용해야 하는 모델이다. 


[2] Trabucco, Brandon, et al. "Conservative objective models for effective offline model-based optimization." International Conference on Machine Learning. PMLR, 2021.

In [129]:
# Proxy Score Function

# Random Forest Regression model
proxy_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Proxy only see part of data
proxy_model.fit(x_train, y_train)
proxy_preds = proxy_model.predict(x_valid)
proxy_mse = mean_squared_error(y_valid, proxy_preds)

print(f"Proxy Regression Validation MSE: {proxy_mse:.4f}")




Proxy Regression Validation MSE: 0.0392


# Conditional Variational AutoEncoder (CVAE)

CVAE는 Genertive model 로서 condition variable $y$ 에 대해서 decision or design variable $x$ 를 generate하는 확률 모델이다. 

즉 CVAE는 $p(x|y,z)$ 를 모델하게 된다. 

구체적은 학습 과정으로는:

1. Encoder 는 $x$와 $y$ 로 부터 평균과 표준편차를 예측하고,
2. Decoder는 Encoder의 평균과 표준편차와 $y$ 값을 input으로 받아서 output으로 $x$를 reconstruct 한다. 

학습 후 사용할 때는 Decoder만 사용하게 되고, 우리가 원하는 정규분포에서 랜덤하게 샘플한 평균, 표준편자, 그리고 원하는 $y*$ 값을 넣어주면, $x$가 생성된다. 


In [146]:
# Define the cVAE model
class cVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, condition_dim):
        super(cVAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim + condition_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )
        
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + condition_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )
        
    def encode(self, x, condition):
        input_condition = torch.cat((x, condition), dim=1)
        hidden = self.encoder(input_condition)
        mu = self.fc_mu(hidden)
        logvar = self.fc_logvar(hidden)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z
    
    def decode(self, z, condition):
        latent_condition = torch.cat((z, condition), dim=1)
        reconstructed = self.decoder(latent_condition)
        return reconstructed

    def forward(self, x, condition):
        mu, logvar = self.encode(x, condition)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decode(z, condition)
        return reconstructed, mu, logvar

# Hyperparameters

input_dim 은 말 그대로 input 되는 변수의 dimension 으로, 우리에게는 feature의 개수가 되겠다. 

hidden_dim 은 encoder와 decoder에서 representation learning 을 할 때의 MLP의 hidden dimension이다. 

lagent_dim 은 conditioning code와 합쳐지기 전의 representation dimension이다. 

condition_dim 은 input condition의 dimension으로 우리는 어떠한 스칼라 값 $y$을 condition으로 주기 때문에 1차원이다.  

optimizer는 adam optimizer를 사용한다. 

In [131]:
# Hyperparameters
input_dim = len(features)
hidden_dim = 256
latent_dim = 64
condition_dim = 1  # Scalar condition

# Create the cVAE model
vae = cVAE(input_dim, hidden_dim, latent_dim, condition_dim)

# Define loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Set up optimizer
optimizer = optim.Adam(vae.parameters(), lr=1e-4)

In [ ]:
# Training CVAE

전형적인 Pytorch training code이다. 

In [132]:
import torch.nn.functional as F
# Example training loop using your dataset and data loaders
def train_vae(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, condition) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.view(-1, input_dim)  # Reshape data if needed
        condition = condition.view(-1, condition_dim)  # Reshape condition if needed
        
        recon_batch, mu, logvar = vae(data, condition)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch {}: Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

# Example usage with your dataset and data loaders
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train_vae(epoch)

Epoch 1: Average loss: 6.6852
Epoch 2: Average loss: 5.5476
Epoch 3: Average loss: 5.5453
Epoch 4: Average loss: 5.5434
Epoch 5: Average loss: 5.5424
Epoch 6: Average loss: 5.5418
Epoch 7: Average loss: 5.5414
Epoch 8: Average loss: 5.5411
Epoch 9: Average loss: 5.5410
Epoch 10: Average loss: 5.5407


# Decoding after training

일전에 언급했던 것 같이 결국 학습된 CVAE에서 사용되는 부분은 학습된 디코더이다. 

디코더에게 우리가 원하는 condition_value를 input으로 넣어주면 $x$가 sample 된다. 

condition_value는 자유롭게 정할 수 있다. 

num_sample은 sample의 개수이다. 

In [133]:
# Example function to decode using trained cVAE with a specific condition value
def decode_with_condition(condition_value, num_samples=1):
    vae.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        condition = torch.tensor([[condition_value]] * num_samples, dtype=torch.float32)
        condition = condition.to(device)
        
        z_samples = torch.randn(num_samples, latent_dim).to(device)  # Sample random latent vectors
        
        reconstructed_samples = vae.decode(z_samples, condition)
    return reconstructed_samples

# Example usage of the decode function
condition_value = 11.8  # The condition value you want to query
num_samples = 15  # Number of samples to generate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae.to(device)  # Move the model to the appropriate device

decoded_samples = decode_with_condition(condition_value, num_samples)

# Print or visualize the decoded samples as needed
print(decoded_samples)




tensor([[0.6318, 0.3343, 0.4549, 0.6203, 0.0538, 0.0567, 0.5732, 0.2402, 0.0658,
         0.1409, 0.3677],
        [0.6249, 0.3312, 0.4569, 0.6068, 0.0533, 0.0525, 0.5658, 0.2359, 0.0689,
         0.1421, 0.3721],
        [0.6414, 0.3287, 0.4477, 0.6111, 0.0557, 0.0619, 0.5632, 0.2431, 0.0738,
         0.1486, 0.3631],
        [0.6223, 0.3271, 0.4436, 0.6202, 0.0540, 0.0561, 0.5798, 0.2405, 0.0667,
         0.1456, 0.3596],
        [0.6265, 0.3367, 0.4591, 0.6124, 0.0548, 0.0577, 0.5596, 0.2376, 0.0731,
         0.1452, 0.3597],
        [0.6329, 0.3364, 0.4546, 0.6187, 0.0538, 0.0567, 0.5793, 0.2339, 0.0757,
         0.1387, 0.3564],
        [0.6288, 0.3307, 0.4493, 0.6105, 0.0541, 0.0568, 0.5717, 0.2415, 0.0685,
         0.1348, 0.3640],
        [0.6194, 0.3344, 0.4489, 0.6205, 0.0566, 0.0552, 0.5739, 0.2306, 0.0714,
         0.1405, 0.3602],
        [0.6379, 0.3317, 0.4595, 0.6213, 0.0608, 0.0587, 0.5730, 0.2366, 0.0696,
         0.1455, 0.3672],
        [0.6277, 0.3272, 0.4547, 0.61

# Oracle Test

우리의 역생성모델이 만든 $x$가 얼마나 잘 만들어졌는지 검증하기 위해서 oracle model을 마지막으로 이용하여 해당 값을 유추한다. 

In [137]:
if torch.cuda.is_available():
    decoded_samples = decoded_samples.cpu()

print(oracle_model.predict(decoded_samples).mean())

12.163513333333333


/home/as12088/anaconda3/envs/hkt/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


# Renormalization

다시 원래의 데이터 형태로 돌리기 위해서 renormalization 을 수행한다. 

In [143]:
# Renormalize a new value x using the same scaling parameters
def renormalize_back(x_normalized):
    return x_normalized * (df.max() - df.min() ) + df.min() 


outputs = pd.DataFrame(decoded_samples, columns = features)
outputs = renormalize_back(outputs)
print(outputs)

    BUILDING_Tread 폭 - 실측  BUILDING_Tread 길이 - 실측  BUILDING_Belt #1 - 폭   \
0              260.484223             2007.837882             221.641014   
1              260.177878             2007.599909             221.674440   
2              260.907645             2007.410595             221.519968   
3              260.062075             2007.290422             221.450971   
4              260.247607             2008.026912             221.710645   
5              260.531129             2008.003160             221.634859   
6              260.352236             2007.568202             221.546602   
7              259.936145             2007.849069             221.538705   
8              260.755020             2007.641769             221.718168   
9              260.303081             2007.299259             221.637693   
10             260.595668             2008.066134             221.623051   
11             260.188201             2007.707588             221.794708   
12          